In [1]:
import numpy as np
import opensim as osim
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import os

In [2]:
# Keeping pandas from truncating long strings
pd.set_option('display.max_colwidth', 10000000)

## Paths

In [3]:
os.getcwd()

'c:\\Users\\User\\OneDrive\\Documents\\WORK\\JRF_GaitAnalysis\\modelling_paper\\repo\\folder\\worked_example\\model_update'

In [4]:
# path to generic scaled model
scaled_model = f"../final_results/generic_scaled/scaled_model_joints.osim"

In [5]:
# path to the new personalized model
new_model_name = f'tps_transformed.osim'
path_to_model = os.path.join("..", 'final_results', 'personalized')
if not os.path.exists(path_to_model):
    print(path_to_model)
    os.makedirs(path_to_model)
new_model = os.path.join(path_to_model, new_model_name)

In [6]:
# IMPORTANT : THESE SHOULD BE PERSON-SPECIFIC
mass_text = '52.4'
height_text = '1.60'
age_text = '33'

# path to experimental .trc file : <marker_file>
experimental_markers = r'../motion_lab/static/Static03/task.trc'
static_df = pd.read_csv(experimental_markers, delimiter='\t', skiprows=3, header=[0,1], index_col=0)

In [7]:
# path to TPS results
tps_folder = os.path.join("..", 'final_results')

## Update Scaled Model with TPS data

### Import Transformed Markers

In [8]:
markers_df = pd.concat([pd.read_csv(os.path.join(tps_folder, 'pelvis_bone_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'femur_l_bone_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'femur_r_bone_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'patella_l_bone_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'patella_r_bone_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'tibia_l_bone_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'tibia_r_bone_markers.csv'), index_col=0)])


In [9]:
markers_df = markers_df.set_index('name')
markers_df.to_csv(os.path.join(tps_folder, 'markers_transformed.csv'))

### Import Transformed Muscle Paths

In [10]:
mscles_df = pd.concat([pd.read_csv(os.path.join(tps_folder, 'pelvis_muscle_paths.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'femur_l_muscle_paths.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'femur_r_muscle_paths.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'patella_l_muscle_paths.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'patella_r_muscle_paths.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'tibia_l_muscle_paths.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'tibia_r_muscle_paths.csv'), index_col=0)])
mscles_df = mscles_df.set_index('name')

### Import Transformed Wrapping Surfaces

In [11]:
wrp_df  = pd.concat([pd.read_csv(os.path.join(tps_folder, 'pelvis_wrap_translations.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'femur_l_wrap_translations.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'femur_r_wrap_translations.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'tibia_l_wrap_translations.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'tibia_r_wrap_translations.csv'), index_col=0)])
wrp_df = wrp_df.set_index('name')
wrp_df.to_csv(os.path.join(tps_folder, 'wrp_transformed.csv'))

### Import transformed Skin Markers

In [12]:
skin_df = pd.concat([pd.read_csv(os.path.join(tps_folder, 'pelvis_skin_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'femur_l_skin_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'femur_r_skin_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'tibia_l_skin_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'tibia_r_skin_markers.csv'), index_col=0)])
skin_df = skin_df.set_index('name')
skin_df.to_csv(os.path.join(tps_folder, 'skin_transformed.csv'))

### Parse current scaled model

In [13]:
tree=ET.parse(scaled_model)
root = tree.getroot()

### Update reference geometry

In [14]:
ref_geom_dic = {'body': [], 'Mesh name' : [], 'Mesh file' : []}

ref_geom_dic['body'].append("pelvis"); ref_geom_dic['Mesh name'].append("pelvis_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\r_pelvis.stl'))
ref_geom_dic['body'].append("pelvis"); ref_geom_dic['Mesh name'].append("pelvis_geom_2"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\l_pelvis.stl'))
ref_geom_dic['body'].append("pelvis"); ref_geom_dic['Mesh name'].append("pelvis_geom_3"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\sacrum.stl'))

ref_geom_dic['body'].append("femur_r"); ref_geom_dic['Mesh name'].append("femur_r_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\femur_r.stl'))
ref_geom_dic['body'].append("femur_l"); ref_geom_dic['Mesh name'].append("femur_l_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\femur_l.stl'))

ref_geom_dic['body'].append("tibia_r"); ref_geom_dic['Mesh name'].append("tibia_r_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\tibia_r.stl'))
ref_geom_dic['body'].append("tibia_r"); ref_geom_dic['Mesh name'].append("tibia_r_geom_2"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\fibula_r.stl'))

ref_geom_dic['body'].append("tibia_l"); ref_geom_dic['Mesh name'].append("tibia_l_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\tibia_l.stl'))
ref_geom_dic['body'].append("tibia_l"); ref_geom_dic['Mesh name'].append("tibia_l_geom_2"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\fibula_l.stl'))

ref_geom_dic['body'].append("patella_r"); ref_geom_dic['Mesh name'].append("patella_r_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\patella_r.stl'))
ref_geom_dic['body'].append("patella_l"); ref_geom_dic['Mesh name'].append("patella_l_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\patella_l.stl'))

In [15]:
ref_geom_df = pd.DataFrame(ref_geom_dic).set_index('Mesh name')

In [16]:
for Mesh in root.iter('Mesh'):
    point = Mesh.attrib['name']
    if point in ref_geom_df.index:
        new_text = ref_geom_df.loc[point, 'Mesh file']
        # print(point, new_text)
        Mesh.find('mesh_file').text = new_text
        Mesh.find('scale_factors').text = '1 1 1'
    else: pass

### Update muscle paths

In [17]:
muscle_path_names = []
muscle_path_old = []
muscle_path_new = []

for PathPoint in root.iter('PathPoint'):
    point = PathPoint.attrib['name']   
    if point in mscles_df.index:      
        
        location = mscles_df.loc[point, 'location']
        new_text = location[1:-1]
        muscle_path_names.append(point)
        muscle_path_new.append(new_text)
        muscle_path_old.append(PathPoint.find('location').text)

        PathPoint.find('location').text = new_text
    else: pass

In [18]:
muscle_path_new_df = pd.DataFrame([[float(i) for i in point.split()] for point in muscle_path_new], index=muscle_path_names, columns=['x','y','z'])
muscle_path_old_df = pd.DataFrame([[float(i) for i in point.split()] for point in muscle_path_old], index=muscle_path_names, columns=['x','y','z'])

muscle_path_diff_df = muscle_path_old_df - muscle_path_new_df
muscle_path_diff_df['d'] = (muscle_path_diff_df['x']**2+muscle_path_diff_df['y']**2+muscle_path_diff_df['z']**2)**0.5

### Update muscle wrapping surfaces

In [19]:
wrap_cyl_name = []
wrap_transl_old = []
wrap_transl_new = []
wrap_radius_old = []
wrap_radius_new = []

In [20]:
for WrapCylinder in root.iter('WrapCylinder'):
    point = WrapCylinder.attrib['name']
    wrap_cyl_name.append(point)

    if point in wrp_df.index:
        translation = wrp_df.loc[point, 'location']
        wrap_transl_new.append(translation)
        wrap_transl_old.append(WrapCylinder.find('translation').text)
        WrapCylinder.find('translation').text = translation[1:-1]

    else: pass

### Update Translation of joint centres

In [21]:
torso_origin_in_pelvis = markers_df.loc['torso_origin_in_pelvis', 'location']

femur_r_center_in_pelvis = markers_df.loc['femur_r_center_in_pelvis', 'location']
femur_l_center_in_pelvis = markers_df.loc['femur_l_center_in_pelvis', 'location']
knee_l_center_in_femur = markers_df.loc['knee_l_center_in_femur_l', 'location']
knee_r_center_in_femur = markers_df.loc['knee_r_center_in_femur_r', 'location']
patella_l_center_in_femur = markers_df.loc['patella_l_in_femur_l', 'location']
patella_r_center_in_femur = markers_df.loc['patella_r_in_femur_r', 'location']

talus_l_center_in_tibia = markers_df.loc['talus_l_center_in_tibia', 'location']
talus_r_center_in_tibia = markers_df.loc['talus_r_center_in_tibia', 'location']

In [22]:
joint_centers_names = []
joint_centers_new = []
joint_centers_old = []

for joint in root.iter('CustomJoint'):
    
    name = joint.attrib['name']
    if joint.attrib['name'] ==  "back":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "pelvis_offset":
                new_text = torso_origin_in_pelvis[1:-1]
                joint_centers_names.append("back")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text

            else: pass

    if joint.attrib['name'] ==  "hip_r":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "pelvis_offset":                
                new_text = femur_r_center_in_pelvis[1:-1]
                joint_centers_names.append("hip_r")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)           
                frame.find('translation').text = new_text

            else: pass
    
    elif joint.attrib['name'] ==  "hip_l":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "pelvis_offset":
                new_text = femur_l_center_in_pelvis[1:-1]
                joint_centers_names.append("hip_l")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text
            else: pass
            
    elif joint.attrib['name'] ==  "walker_knee_r":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "femur_r_offset":
                new_text = knee_r_center_in_femur[1:-1]
                joint_centers_names.append("walker_knee_r")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text

            #if frame.attrib['name'] == "tibia_r_offset":
            #    new_text = f"{walker_knee_r_tibia_r_offset[0]}, {walker_knee_r_tibia_r_offset[1]}, {walker_knee_r_tibia_r_offset[2]}"
            #    frame.find('translation').text = new_text
            else: pass    
    
    elif joint.attrib['name'] ==  "walker_knee_l":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "femur_l_offset":
                new_text = knee_l_center_in_femur[1:-1]
                joint_centers_names.append("walker_knee_l")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text

    #        if frame.attrib['name'] == "tibia_l_offset":
    #            new_text = f"{walker_knee_l_tibia_l_offset[0]}, {walker_knee_l_tibia_l_offset[1]}, {walker_knee_l_tibia_l_offset[2]}"
    #            frame.find('translation').text = new_text
            else: pass

    elif joint.attrib['name'] ==  "patellofemoral_r":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "femur_r_offset":               
                new_text = knee_r_center_in_femur[1:-1]
                joint_centers_names.append("patellofemoral_r")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text
           else: pass
    
    elif joint.attrib['name'] ==  "patellofemoral_l":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "femur_l_offset":              
                new_text = knee_l_center_in_femur[1:-1]
                joint_centers_names.append("patellofemoral_l")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text
           else: pass

for joint in root.iter('PinJoint'):   
    name = joint.attrib['name']
    if joint.attrib['name'] ==  "ankle_r":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "tibia_r_offset":              
                new_text = talus_r_center_in_tibia [1:-1]
                joint_centers_names.append("ankle_r")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text
           else: pass
    
    elif joint.attrib['name'] ==  "ankle_l":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "tibia_l_offset":              
                new_text = talus_r_center_in_tibia [1:-1]              
                joint_centers_names.append("ankle_r")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)              
                frame.find('translation').text = new_text
           else: pass
    else: pass

### Update Markers for Joint Positions in Parent

In [23]:
m_names = ['torso_origin_in_pelvis','femur_l_center_in_pelvis', 'femur_r_center_in_pelvis',  'knee_l_center_in_femur_l', 'patella_l_in_femur_l', 'knee_r_center_in_femur_r', 'patella_r_in_femur_r']

for marker in root.iter('Marker'):
    name = marker.attrib['name']
    if name == 'ankle_l_tibia_l_offset':
        old_text = marker.find('location').text
        new_text = markers_df.loc['talus_l_center_in_tibia', 'location']
        marker.find('location').text = new_text[1:-1]
    elif name == 'ankle_r_tibia_r_offset':
        old_text = marker.find('location').text
        new_text = markers_df.loc['talus_r_center_in_tibia', 'location']
        marker.find('location').text = new_text[1:-1]
    elif name in m_names:
        old_text = marker.find('location').text
        new_text = markers_df.loc[name, 'location']
        marker.find('location').text = new_text[1:-1]

### Uldate Skin Markers

In [24]:
for marker in root.iter('Marker'):
    name = marker.attrib['name']
    if name in list(skin_df.index):
        old_text = marker.find('location').text
        new_text = skin_df.loc[name, 'location']
        marker.find('location').text = new_text[1:-1]
    else: pass

### Export transformed model

In [25]:
tree.write(new_model)

## Scale muscle fibres

In [26]:
# from fibre_scale_script import *

In [27]:
# # define models
# generic_model = '../templates/RajagopalModified_generic_copy.osim'
# model_to_update = f'final_results/tps_transformed.osim'
# updated_model = f'final_results/tps_fibres_updated.osim'

# # run scripts
# osimModel_opt, SimInfo = optimMuscleParams(generic_model, model_to_update, 2, 'final_results/logging')

# # printing the optimized model
# osimModel_opt.printToXML(updated_model)

## Move bone surfaces to the new model folder

In [28]:
bones_folder = os.path.join(path_to_model, 'bones')
if not os.path.exists(bones_folder):
    print(bones_folder)
    os.makedirs(bones_folder)

In [ ]:
# this code copies the bones from the TPS folder to the bones folder

import shutil

# Create the bones folder if it doesn't exist
os.makedirs(bones_folder, exist_ok=True)

for file in os.listdir(tps_folder):
    if file.endswith('.stl'):
        old_path = os.path.join(tps_folder, file)
        new_path = os.path.join(bones_folder, file)
        print(old_path, new_path)
        shutil.copy(old_path, new_path)

..\final_results\femur_l.stl ..\final_results\personalized\bones\femur_l.stl
..\final_results\femur_r.stl ..\final_results\personalized\bones\femur_r.stl
..\final_results\fibula_l.stl ..\final_results\personalized\bones\fibula_l.stl
..\final_results\fibula_r.stl ..\final_results\personalized\bones\fibula_r.stl
..\final_results\l_pelvis.stl ..\final_results\personalized\bones\l_pelvis.stl
..\final_results\patella_l.stl ..\final_results\personalized\bones\patella_l.stl
..\final_results\patella_r.stl ..\final_results\personalized\bones\patella_r.stl
..\final_results\r_pelvis.stl ..\final_results\personalized\bones\r_pelvis.stl
..\final_results\sacrum.stl ..\final_results\personalized\bones\sacrum.stl
..\final_results\tibia_l.stl ..\final_results\personalized\bones\tibia_l.stl
..\final_results\tibia_r.stl ..\final_results\personalized\bones\tibia_r.stl


## Fit skin markers

<div class="alert alert-block alert-info">
<b>Before the final Skin Marker Fitting</b> please open model "final_results/tps_transformed.osim". Save the skin landmarks in a 'markers.xml' file.

In [31]:
## Fit Skin Markers
# <model_file>
model = new_model # f'final_results/tps_fibres_updated.osim'

# <time_range>'
time_range = f' 0 {list(static_df.loc[static_df.shape[0], "Time"])[0] }'

# <output_scale_file>
path_to_model = '../final_results/personalized'
output_scale_file = os.path.join(path_to_model, 'marker_placer.txt')
output_scaling_settings = os.path.join(path_to_model, f'marker_placer.xml')
output_model_file = os.path.join(path_to_model, f'tps_skin_updated.osim')

# <marker_set_file>
path_to_marker_set = os.path.join(path_to_model, 'markers.xml')

In [32]:
# C:\Users\User\OneDrive\Documents\WORK\JRF_GaitAnalysis\modelling_paper\repo\folder\final_results\personalized\markers.xml

In [ ]:
# parse the .xml with markers in body frames
scaling_tree=ET.parse("../templates/marker_placer.xml")
scaling_root = scaling_tree.getroot()

for generic_model in scaling_root.iter('model_file'):
    generic_model.text = model
for generic_marker_set in scaling_root.iter('marker_set_file'):
    generic_marker_set.text = path_to_marker_set

for exp_markers in scaling_root.iter('marker_file'):
    exp_markers.text = experimental_markers

for time in scaling_root.iter('time_range'):
    time.text = time_range
for output in scaling_root.iter('output_model_file'):
    output.text = output_model_file
for scale in scaling_root.iter('output_scale_file'):
    scale.text = output_scale_file

for time in scaling_root.iter('time_range'):
    time.text = time_range
for output in scaling_root.iter('output_model_file'):
    output.text = output_model_file
for scale in scaling_root.iter('output_scale_file'):
    scale.text = output_scale_file

In [34]:
scaling_tree.write(output_scaling_settings)

In [35]:
from simFunctions import runProgram

cmdprog = 'opensim-cmd'
cmdtool = 'run-tool'
cmdfile = output_scaling_settings
cmdfull = [cmdprog, cmdtool, cmdfile]
rc = runProgram(cmdfull)

c:\Users\User\miniconda3\envs\py311\Lib\subprocess.py:1016: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)


[info] Preparing to run ScaleTool.

[info] Processing subject Rajagopal-generic-scaled-scaled...

[info] Step 1: Loading generic model

[info] Loaded model Rajagopal-generic-scaled from file c:\Users\User\OneDrive\Documents\WORK\JRF_GaitAnalysis\modelling_paper\repo\folder\worked_example\final_results\personalized\tps_transformed.osim

[warning] Couldn't find file 'r_talus.vtp'.

[warning] Couldn't find file 'r_foot.vtp'.

[warning] Couldn't find file 'r_bofoot.vtp'.

[warning] Couldn't find file 'l_talus.vtp'.

[warning] Couldn't find file 'l_foot.vtp'.

[warning] Couldn't find file 'l_bofoot.vtp'.

[warning] Couldn't find file 'hat_spine.vtp'.

[warning] Couldn't find file 'hat_jaw.vtp'.

[warning] Couldn't find file 'hat_skull.vtp'.

[warning] Couldn't find file 'hat_ribs_scap.vtp'.

[info] Loading marker set from '../final_results/personalized\markers.xml'.

[info] Updated markers in model Rajagopal-generic-scaled

[error] Scaling parameters disabled (apply is false) or not set. Mo

## Update Wrapping Surfaces

In [36]:
from wrap_scripts import *

model_to_update = output_model_file
updated_model = os.path.join(path_to_model, 'tps_skin_wrp_updated.osim')

# function to extract muscle paths, wrapping surfaces info and joint info from an osim model
# returns three dataframes: muscles, surfaces and joints
fix_wraps = FixWraps(model_to_update)

PECT_at_femshaft_r is NOT present
PECT_at_femshaft_l is NOT present


In [37]:
wrp_adjust_dict = fix_wraps.collect_wrp_details()
wrp_adjust_df = pd.DataFrame.from_dict(wrp_adjust_dict)
wrp_adjust_df.set_index(pd.Index(['body', 'transl', 'rad', 'point_label', 'point', 'point_body', 'joint_location', 'joint_location+point', 'dist_to_transl',  'dist']), inplace = True)


In [38]:
wrp_adjust_df

,KnExt_at_fem_r,AMprox_at_femshaft_r,GR_at_condyles_r,SM_at_condyles_r,ST_at_condyles_r,KnExt_at_fem_l,AMprox_at_femshaft_l,GR_at_condyles_l
body,femur_r,femur_r,tibia_r,tibia_r,tibia_r,femur_l,femur_l,tibia_l
transl,"[0.00317880787659607, -0.37956007240335693, 0.00189316623328431]","[0.01843032691577612, -0.07007769611499623, 0.00187476127501147]","[-0.0002038201949223, -0.01865194797071473, -0.00448544266317864]","[0.00169926446686766, -0.0184897596316406, -0.00461911117565189]","[0.00075741833241223, -0.01901348250318577, -0.00453156489752562]","[0.0033445005748296, -0.3870226242345994, -0.00151318916820757]","[0.02394465736207983, -0.07125573924272643, -0.00105791526285148]","[0.00020920741203456, -0.01846583876757035, 0.00462866573573066]"
rad,0.022032,0.018595,0.036301,0.035495,0.042856,0.022025,0.018589,0.034719
point_label,vasmed_r-P4,addmagProx_r-P2,grac_r-P2,semimem_r-P2,semiten_r-P2,vasmed_l-P4,addmagProx_l-P2,grac_l-P2
point,"[0.00435350267995259, 0.00059915631419233, -0.00826284302773272]","[0.00177658024709527, -0.07195394721641953, 0.0059718055552296]","[-0.01501567726877589, -0.04357968976846865, -0.02846631781830914]","[-0.02445157191691852, -0.03945240805047948, -0.01803947163861916]","[-0.02754648231138122, -0.04773655112484198, -0.02028074813240684]","[0.00501033358174416, 0.00018591611272888, 0.00641904630252117]","[0.0088764769004826, -0.07391144151240157, -0.00427172711358879]","[-0.01735145000022968, -0.0425791452302602, 0.03389547831582641]"
point_body,patella_r,femur_r,tibia_r,tibia_r,tibia_r,patella_l,femur_l,tibia_l
joint_location,"[8.592514684046e-05, 0.38064508392695484, 0.00129662134372074]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[2.90856079676e-06, 0.3877017931761645, -0.00128970220316093]","[0, 0, 0]","[0, 0, 0]"
joint_location+point,"[0.00443942782679305, 0.38124424024114717, -0.00696622168401198]","[0.00177658024709527, -0.07195394721641953, 0.0059718055552296]","[-0.01501567726877589, -0.04357968976846865, -0.02846631781830914]","[-0.02445157191691852, -0.03945240805047948, -0.01803947163861916]","[-0.02754648231138122, -0.04773655112484198, -0.02028074813240684]","[0.00501324214254092, 0.3878877092888934, 0.00512934409936024]","[0.0088764769004826, -0.07391144151240157, -0.00427172711358879]","[-0.01735145000022968, -0.0425791452302602, 0.03389547831582641]"
dist_to_transl,0.011515,0.017253,0.037628,0.036103,0.043292,0.009384,0.015634,0.04179
dist,0.001306,0.016976,0.03066,0.033854,0.041023,0.001844,0.015385,0.032156


In [39]:
#print points whose location is too far into radius for an automatic adjustment
manual = []
for col in wrp_adjust_df.columns:
    if wrp_adjust_df.loc['dist', col] < wrp_adjust_df.loc['rad', col]-0.01:
        print(col, ':', wrp_adjust_df.loc['point_label', col])
        manual.append(col)

# ignore patella points for now
# adjust other points or wrapping surfaces manually for the input model and run the script again

KnExt_at_fem_r : vasmed_r-P4
KnExt_at_fem_l : vasmed_l-P4


In [40]:
# # update model
tree_model = ET.parse(model_to_update)
root = tree_model.getroot()

wrp_to_change = [i for i in wrp_adjust_df.columns if i not in manual]

for WrapCylinder in root.iter('WrapCylinder'):
    name = WrapCylinder.attrib['name']
    if name in wrp_to_change:
        print(name)
        radius = wrp_adjust_df.loc['rad', name]
        print('old_radius', radius)
        new_rad = wrp_adjust_df.loc['dist', name]
        print('new_radius', new_rad)
        radius_new_text = str(new_rad-0.01)
        WrapCylinder.find('radius').text = radius_new_text

tree_model.write(updated_model)

AMprox_at_femshaft_r
old_radius 0.01859519221338005
new_radius 0.016976170733013007
GR_at_condyles_r
old_radius 0.03630133624352623
new_radius 0.03066019278414282
SM_at_condyles_r
old_radius 0.035494639882558986
new_radius 0.03385425660092008
ST_at_condyles_r
old_radius 0.042855744176385144
new_radius 0.04102255323635481
AMprox_at_femshaft_l
old_radius 0.018589398106118232
new_radius 0.015384597596978722
GR_at_condyles_l
old_radius 0.03471898494989993
new_radius 0.032156486549388304
